In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
#from __future__ import division
%matplotlib inline
import gc
import collections

In [ ]:
sample = np.random.exponential(scale=1/3, size=100)
sns.distplot(list(sample), kde=False, norm_hist=True)
x = np.linspace(0, sample.max(), 1000)
plt.plot(x, 3*np.exp(-3*x))

In [ ]:
def g_j(j, x):
    return j*np.exp(-j*x)

In [ ]:
def likelihood(M, w, data):
    return np.array([(w*g_j(np.arange(1, M+1), x)).sum() for x in data]).prod()

In [ ]:
def draw_d(w, M, x):
    '''
    :type w: np.array
    :type M: int
    :type x: float
    :rtype: int
    '''
    assert len(w)==M
    pr = w*g_j(np.arange(1, M+1), x)
    pr /= pr.sum()
    res = np.random.choice(M, 1, p=pr)
    return res

In [ ]:
def draw_w(M, d, X):
    assert len(d)==len(X)
    w_alpha = np.arange(M)
    cnt = collections.Counter(w_alpha)
    cnt += collections.Counter(d)
    return np.random.dirichlet(list(cnt.values()))

In [ ]:
def M_up(M, w, data):
    p_accept = 1/likelihood(M, w, data)
    #split w
    idx = np.random.randint(M)
    w_split = np.random.beta(2, 2)
    w = np.append(w, w[idx]*(1-w_split))
    w[idx] *= w_split
    #acceptance rate
    p_accept *= likelihood(M+1, w, data)/6/(w_split*(1-w_split))
    return p_accept, w

In [ ]:
def M_down(M, w, data):
    p_accept = 1/likelihood(M, w, data)
    #combine two w
    w_split = np.random.beta(2, 2)
    idx = np.random.randint(M-1)
    w[idx] += w[-1]
    w = w[:-1]
    #idx1, idx2 = np.random.choice(M, 2, replace=False)
    #w[idx1] += w[idx2]
    #w = np.delete(w, idx2)
    #acceptance rate
    p_accept *= likelihood(M-1, w, data)*6*(w_split*(1-w_split))
    return p_accept, w

In [ ]:
def draw_M(M, w, data):
    u = np.random.uniform()
    p, w_up = M_up(M, w, data)
    if M == 1: # M can only go up in this case
        if u < p:
            return M+1, w_up
        else:
            return M, w
    
    if np.random.randint(2): # propose to go up
        if u<p: # accepted to go up
            return M+1, w_up
        else: # stay
            return M, w
    else:
        p, w_down = M_down(M, w, data)
        if u<p: # accepted to go down
            return M-1, w_down
        else: # stay
            return M, w


In [ ]:
M = 1
N = 10000
w = np.random.dirichlet(np.ones(M))
sample2 = []
Ms = []
indices = []
w_list = []
for i in range(N):
    if i%1000 == 0:
        print(i)
    d = np.array([draw_d(w, M, x)[0] for x in sample])
    w = draw_w(M, d, sample)
    w_list.append(w)
    idx = np.random.choice(M, 1, p=w)[0]
    indices.append(idx)
    sample2.append(np.random.exponential(scale=1/(idx+1)))
    Ms.append(M)
    M, w = draw_M(M, w, sample)    

In [ ]:
plt.plot(Ms)

In [ ]:
sns.distplot(Ms, kde=False, norm_hist=True, bins=range(1, 36))

In [ ]:
sns.distplot(sample2, kde=False, norm_hist=True, label='sample')
x = np.linspace(0, max(sample2), 1000)
plt.plot(x, 3*np.exp(-3*x), label='3*exp(-3*x)')
plt.legend()
plt.title('Sampled distribution from Gibbs sampler')

In [ ]:
plt.plot(indices)

In [ ]:
sns.distplot(indices, kde=False, norm_hist=True, bins=np.arange(35))

In [ ]:
sample_mean = [np.mean(sample2[:i]) for i in range(1, N+1)]

In [ ]:
plt.plot(range(1, N+1), sample_mean)

In [ ]:
sns.distplot(sample2[:2000], kde=False, norm_hist=True)
x = np.linspace(0, max(sample2), 1000)
plt.plot(x, 3*np.exp(-3*x))

In [ ]:
sample.mean(), np.mean(sample2)

In [ ]:
gc.collect()

In [ ]:
X = np.linspace(0, 5, 101)

In [ ]:
def computeDistribution(x, w_list):
    n = len(w_list)
    return (w_list*g_j(np.arange(1, n+1), x)).sum()

In [ ]:
y = X*0
for w in w_list:
    y += np.array([computeDistribution(x, w) for x in X])
y /= N

In [ ]:
plt.plot(X, y, label = 'gp')
plt.plot(X, 3*np.exp(-3*X), '--', label='3*exp(-3*x)')
plt.legend()
plt.title('Sampled distribution')

In [ ]:
plt.plot(X, (y-3*np.exp(-3*X))/3*np.exp(-3*X))

In [ ]:
w